In [21]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
import glob
import seaborn as sns

In [22]:
def add_plot_number(gdf):
    gdf = gdf.copy()
    gdf['plot_number'] = np.nan
    
    for index, row in gdf.iterrows():
        fieldID = int(row['fieldID'])
        range_num = 19 - (fieldID - 1) // 24
        col_num = (fieldID - 1) % 24 + 1
        plot_number = f"{range_num:02}{col_num:02}"
        gdf.at[index, 'plot_number'] = plot_number.zfill(4)
    
    return gdf

# Data

In [23]:
result_list = []

for csv in glob.glob('./outputs/dep/*.csv'):
    mon, day, year = os.path.basename(csv).split('_')[:3]
    date = '-'.join([year, mon.zfill(2), day.zfill(2)])
    temp_df = pd.read_csv(csv)
    temp_df = temp_df.rename(columns={'ID': 'fieldID'})
    temp_df['fieldID'] = temp_df['fieldID'].astype(int)
    temp_df = temp_df.sort_values('fieldID')
    temp_df['fieldID'] = list(range(1, 457))
    temp_df['date'] = date

    temp_df = add_plot_number(temp_df)
    temp_df['plot_number'] = temp_df['plot_number'].astype(int).astype(str).str.zfill(4)
    temp_df = temp_df.drop('fieldID', axis=1).rename(columns={'plot_number': 'fieldID'})
    result_list.append(temp_df)

result_df = pd.concat(result_list)
result_df['date'] = pd.to_datetime(result_df['date'])
result_df = result_df.rename(columns={'fieldID': 'plot'})
result_df = result_df.sort_values(['date', 'plot'])
result_df = result_df.set_index(['date', 'plot'])

In [24]:
result_df.to_csv('F120_beans_2023.csv')

# Shapefile

In [ ]:
# def add_plot_number(gdf):
#     gdf = gdf.copy()
#     gdf['plot_number'] = np.nan
    
#     for index, row in gdf.iterrows():
#         fieldID = row['fieldID']
#         range_num = 20 - (fieldID - 1) // 24
#         col_num = (fieldID - 1) % 24 + 1
#         plot_number = f"{range_num:02}{col_num:02}"
#         gdf.at[index, 'plot_number'] = plot_number.zfill(4)
    
#     return gdf




gdf = gpd.read_file("shapefile/F120_beans_2023_rotated_qgis.shp")
gdf['fieldID'] = gdf['fieldID'].astype(int)
gdf = gdf.sort_values('fieldID')
gdf['fieldID'] = list(range(1, 457))

gdf = add_plot_number(gdf)
gdf['plot_number'] = gdf['plot_number'].astype(int).astype(str).str.zfill(4)
gdf = gdf.drop('fieldID', axis=1).rename(columns={'plot_number': 'fieldID'})
# gdf.to_file("shapefile/F120_beans_2023_rotated_qgis_updated.shp")